# Time Travel Task Analyses

This notebook contains the analyses necessary to generate the results for the Time Travel Task. The time travel task outputs a binary format log file for space efficiency which first needs to be converted to the appropriate intermediate representations (the iPosition format for test results and an intermediate navigation format for study/test navigation results).

Note: This file was generated from other, older analysis files and needs to be tested.

## Data Conversion

First we'll process the data into the appropriate formats.

In [3]:
import cogrecon.core.data_flexing.time_travel_task.time_travel_task_to_iposition as ttt2i

In [ ]:
data_directory = r'\\192.168.2.2\kevroy314\Work\Time Travel Task\Paper Data (cleaned)'
data_output_directory = r'\\192.168.2.2\kevroy314\Work\Time Travel Task\Paper Data (iPosition format cleaned)'

ttt2i.time_travel_task_to_iposition(data_directory, data_output_directory)

## Test Analyses

For test, there are several primary analyses to be run. First, a basic analysis for the purpose of determining the test-time performance in space or in time are performed. This can be done quite simply with the basic batch processing functions.

Note that these files contain all the output metrics necessary to perform the statistics on Space/Time Misplacement, Number of Incorrect Event Types, Miassignments in Space vs. Time, and Context Boundary Effects.

In [2]:
from cogrecon.core.batch_pipeline import batch_pipeline
from cogrecon.core.data_flexing.time_travel_task.time_travel_task_analytics import summarize_test_data

In [ ]:
summarize_test_data(search_directory=data_directory)  # For Time Travel Task specific analyses
# For Time vs. Space Reconstruction Analyses
temporal_result = batch_pipeline(str(data_output_directory), 'TimeTravelTask_TimeOnly.csv', removal_dim_indicies=[0, 1])
spatial_result = batch_pipeline(str(data_output_directory), 'TimeTravelTask_SpaceOnly.csv', removal_dim_indicies=[2])

## Basic Navigation Analyses

Next, we can create the summary file for basic navigation metrics.

Note that this file contains all the output metrics for basic spatial and temporal navigation.

In [5]:
from cogrecon.core.data_flexing.time_travel_task.time_travel_task_analytics import summarize_navigation_data

In [ ]:
summarize_navigation_data(search_directory=data_directory)  # For Time Travel Task specific analyses

## Effect of Context on Misassignment

The analysis of the effect of context on misassignment is not included in any of the packages directly. As a result, we need to do some custom computation to get out these numbers.

First, we need to read the data from the **Accurate Misassignment Pairs** column in the temporal only test output file. Because we're running this in one file, we can simply extract this from the output result rather than reading from file.

In [6]:
from cogrecon.core.batch_pipeline import get_header_labels

In [ ]:
misassignment_pairs = np.transpose(temporal_result)[get_header_labels().index('Accurate Misassignment Pairs')]

Next we'll process the list, counting the number of within vs. across context pairs.

In [ ]:
import numpy as np

# The pairs which share a context (note that order doesn't matter for this)
within_key = [[0, 1], [1, 0], [2, 3], [3, 2], [4, 5], [5, 4], [6, 7], [7, 6]]
# The items to exclude because they had no contextual information
# thus if they were given temporal information, they would not be a valid misassignment
exclusion_items = [8, 9]

within_list = []
across_list = []
totals_list = []
for i, a in enumerate(misassignment_pairs):
    totals_list.append(len(a))
    within_list.append(0)
    across_list.append(0)
    for el in a:
        if all([el_i not in exclusion_items for el_i in el]):
            if el in within_key:
                within_list[-1] += 1
            else:
                across_list[-1] += 1
within_list_proportion = [float(x)/float(y) if y is not 0 else np.nan for x, y in zip(within_list, totals_list)]
across_list_proportion = [float(x)/float(y) if y is not 0 else np.nan for x, y in zip(across_list, totals_list)]

Next we can save these results to an intermediate file for running statistics.

In [ ]:
with open('misassignments_by_context.csv', 'w') as fp:
    fp.write('total_misassignments,within_misassignments,across_misassignments,within_misassignment_proportions,across_misassignment_proportions\n')
    for t, w, a, wp, ap in zip(totals_list,within_list ,across_list,within_list_proportion,across_list_proportion):
        fp.write('{0},{1},{2},{3},{4}\n'.format(t, w, a, wp, ap))